In [321]:
using LowLevelFEM
import LowLevelFEM as FEM
gmsh.initialize()

In [322]:
gmsh.open("test-twist-3D.geo")

Info    : Reading 'test-twist-3D.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 40%] Meshing curve 2 (Line)
Info    : [ 70%] Meshing curve 3 (Circle)
Info    : Done meshing 1D (Wall 0.000491927s, CPU 0.000451s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Cylinder, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 70%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0474216s, CPU 0.037664s)
Info    : Meshing 3D...
Info    : 3D Meshing 1 volume with 1 connected component
Info    : Tetrahedrizing 351 nodes...
Info    : Done tetrahedrizing 359 nodes (Wall 0.0106513s, CPU 0.00516s)
Info    : Reconstructing mesh...
Info    :  - Creating surface mesh
Info    :  - Identifying boundary edges
Info    :  - Recovering boundary
Info    : Done reconstructing mesh (Wall 0.0229711s, CPU 0.013388s)
Info    : Found volume 1
Info    : It. 0 - 0 nodes created - worst tet radius 2.0

In [323]:
mat = FEM.material("body")
elastic = FEM.Problem([mat])
thermo = FEM.Problem([mat], type=:HeatConduction)

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 302)
Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 302)


LowLevelFEM.Problem("test-twist-3D", :HeatConduction, 3, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 2648)

In [324]:
fx(x, y, z) = -y * 100
fy(x, y, z) = x * 100
load = FEM.load("load", fx=fx, fy=fy, fz=-10)
supp = FEM.displacementConstraint("supp", ux=0, uy=0, uz=0)

("supp", 0, 0, 0)

In [325]:
u = FEM.solveDisplacement(elastic, [load], [supp])

LowLevelFEM.VectorField(Matrix{Float64}[], [-0.00013303957121353413; -1.2996882492884803; … ; -0.3135490877687742; -0.004356539884407498;;], [0.0], Int64[], 1, :u3D)

In [326]:
FEM.showDoFResults(elastic, u, :uvec)

0

In [327]:
T0 = FEM.initialTemperature(thermo, "body", T=273)

LowLevelFEM.ScalarField(Matrix{Float64}[], [273.0; 273.0; … ; 273.0; 273.0;;], [0.0], Int64[], 1, :T)

In [328]:
L = FEM.latentHeatMatrix(thermo, u, u, T0)

2648×2648 SparseArrays.SparseMatrixCSC{Float64, Int64} with 63014 stored entries:
⎡⣿⣿⡀⣰⣿⣿⣳⣶⣦⣀⣀⠀⠀⠀⠀⠀⠀⠀⠀⣟⣧⣶⣿⣷⣒⣖⣶⣶⣗⣶⣶⣶⣶⣶⣷⣇⣻⣿⣿⣿⎤
⎢⢀⣨⣻⣾⣿⣇⣠⣠⣠⣦⣼⡿⠿⣷⣶⣶⣤⣤⣀⠀⣀⣿⣿⣿⣿⣿⣿⣿⣿⣿⣷⣿⣾⣿⣿⣿⣿⣿⣿⣧⎥
⎢⣿⣿⠿⢿⣿⣿⣷⣿⣿⣿⡿⠟⠻⠿⠿⠷⣿⣿⣿⣷⣦⣿⣿⡿⣿⡿⢿⣿⣿⣿⣿⡿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⢹⣾⠀⣺⣽⣿⠻⣦⣳⢿⢃⠀⠀⠀⠀⠀⠈⣐⡮⠉⠀⣿⣇⣶⢖⣵⣤⣆⡞⣤⢶⡧⢞⢾⣿⡿⢻⣿⡻⡿⎥
⎢⠈⢻⠠⣾⣿⣿⣽⣞⣿⣿⣃⠀⠀⠀⢠⠤⣶⢾⣾⠄⠛⣿⡿⣷⣢⣶⣟⣾⡅⣗⢿⣾⣾⢟⣻⢿⢭⣟⣻⣯⎥
⎢⠀⠘⣶⡿⣿⠏⠉⠐⠉⠘⠻⣦⡀⠀⣤⠆⣜⡆⠩⠀⠀⣿⡷⠋⠿⣷⡿⡥⣥⡃⡉⣷⢸⣿⣻⣏⣝⣿⢫⡟⎥
⎢⠀⠀⢿⣧⣿⡆⠀⠀⠀⠀⠀⠈⣿⣿⣇⡀⡠⡫⠄⠀⠀⣿⣷⢰⣾⠂⡰⣿⡻⣟⡇⣾⡼⣿⣿⣻⣽⣪⡷⠅⎥
⎢⠀⠀⢸⣿⢿⡇⠀⠀⠀⡖⠠⠟⠉⠹⣻⣾⣿⣟⣿⠀⠀⣿⣿⣹⡝⡻⣏⢻⣿⣿⣻⠿⣧⣮⣿⣙⣿⣿⢟⡧⎥
⎢⠀⠀⠀⣿⣿⣿⢂⢠⣸⣟⠲⠽⡤⡪⣿⢿⣿⣿⣿⠄⠀⣿⡿⣯⣯⣿⡿⣭⣸⣽⢿⣿⣳⡜⣿⣿⣷⣿⣿⣾⎥
⎢⣤⢤⠀⠘⢿⣿⡎⠋⠚⠟⠃⠂⠀⠁⠛⠛⠛⠟⠻⣦⡄⣿⣯⠗⠺⠓⠚⢻⠏⠞⣼⡗⠛⡬⡟⣽⣿⡿⣷⣷⎥
⎢⢩⣿⣤⣼⣬⣿⣤⣤⣿⣤⣤⣤⣤⣤⣤⣤⣤⣤⣤⣭⣿⣿⣧⣼⣤⣼⣤⣤⣧⣤⣿⣧⣿⣿⣼⣯⣯⣼⣿⣿⎥
⎢⢿⣿⣿⣿⣿⡿⢩⣽⢿⣯⡽⠋⢙⣛⣟⣻⡿⣯⢯⠟⣉⣿⣿⣿⣯⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣽⢿⣟⣯⣻⎥
⎢⢸⢼⣿⣿⣿⡿⢜⣵⢨⣾⢿⣧⠺⠛⣷⡩⣯⣿⢾⠂⣀⣿⣯⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡿⣿⣷⡟⣿⣻⢽⎥
⎢⢸⣿⣿⣿⣿⣷⠠⢿⣻⣽⠟⡯⣴⣮⣯⣙⡟⣯⣾⣀⠀⣿⣿⣿⣿⣿⣿⣿⣏⣿⣿⣿⣿⣿⣿⣿⣿⣿⣻⣭⎥
⎢⢹⣽⣿⣿⣿⣿⠚⣭⢥⢭⠥⠻⣿⢮⣿⣿⣖⣾⣫⠅⠉⣿⣿⣿⣿⣿⣯⣽⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⢸⣿⣽⣿⣿⡿⠼⡷⣻⣷⢧⣬⣩⣭⣿⡞⣿⣷⢶⠿⠿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣷⣿⣿⣿⣿⣿⣿⣞⎥
⎢⢸⣿⣾⣿⣿⣿⣺⣕⣾⢟⣶⣶⣶⣯⡩⣿⣙⠾⡛⡤⣿⣿⣿⣿⣿⡿⣿⣿⣿⣿⣽⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⠽⢿⣿⣿⣿⣿⣿⡿⣿⣞⡿⢾⣿⣻⣟⢻⣿⣿⣟⣭⡶⣿⣟⣿⢿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣾⣿⣿⣿⣿⣿⣶⣧⢷⣷⣽⡳⣻⣿⣿⣽⣿⣿⡿⣋⣿⣿⢷⣿⣭⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎣⣿⣿⠿⣿⣿⣿⣿⡮⡿⣾⣯⠶⠝⠏⠿⡵⣻⣿⢽⣿⣿⣿⣯⣻⣟⣞⡟⣾⣿⣿⣻⢿⣿⣿⣿⣿⣿⣿⣿⣿⎦

In [329]:
C = FEM.heatCapacityMatrix(thermo)

2648×2648 SparseArrays.SparseMatrixCSC{Float64, Int64} with 63014 stored entries:
⎡⣿⣿⡀⣰⣿⣿⣳⣶⣦⣀⣀⠀⠀⠀⠀⠀⠀⠀⠀⣟⣧⣶⣿⣷⣒⣖⣶⣶⣗⣶⣶⣶⣶⣶⣷⣇⣻⣿⣿⣿⎤
⎢⢀⣨⣻⣾⣿⣇⣠⣠⣠⣦⣼⡿⠿⣷⣶⣶⣤⣤⣀⠀⣀⣿⣿⣿⣿⣿⣿⣿⣿⣿⣷⣿⣾⣿⣿⣿⣿⣿⣿⣧⎥
⎢⣿⣿⠿⢿⣿⣿⣷⣿⣿⣿⡿⠟⠻⠿⠿⠷⣿⣿⣿⣷⣦⣿⣿⡿⣿⡿⢿⣿⣿⣿⣿⡿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⢹⣾⠀⣺⣽⣿⠻⣦⣳⢿⢃⠀⠀⠀⠀⠀⠈⣐⡮⠉⠀⣿⣇⣶⢖⣵⣤⣆⡞⣤⢶⡧⢞⢾⣿⡿⢻⣿⡻⡿⎥
⎢⠈⢻⠠⣾⣿⣿⣽⣞⣿⣿⣃⠀⠀⠀⢠⠤⣶⢾⣾⠄⠛⣿⡿⣷⣢⣶⣟⣾⡅⣗⢿⣾⣾⢟⣻⢿⢭⣟⣻⣯⎥
⎢⠀⠘⣶⡿⣿⠏⠉⠐⠉⠘⠻⣦⡀⠀⣤⠆⣜⡆⠩⠀⠀⣿⡷⠋⠿⣷⡿⡥⣥⡃⡉⣷⢸⣿⣻⣏⣝⣿⢫⡟⎥
⎢⠀⠀⢿⣧⣿⡆⠀⠀⠀⠀⠀⠈⣿⣿⣇⡀⡠⡫⠄⠀⠀⣿⣷⢰⣾⠂⡰⣿⡻⣟⡇⣾⡼⣿⣿⣻⣽⣪⡷⠅⎥
⎢⠀⠀⢸⣿⢿⡇⠀⠀⠀⡖⠠⠟⠉⠹⣻⣾⣿⣟⣿⠀⠀⣿⣿⣹⡝⡻⣏⢻⣿⣿⣻⠿⣧⣮⣿⣙⣿⣿⢟⡧⎥
⎢⠀⠀⠀⣿⣿⣿⢂⢠⣸⣟⠲⠽⡤⡪⣿⢿⣿⣿⣿⠄⠀⣿⡿⣯⣯⣿⡿⣭⣸⣽⢿⣿⣳⡜⣿⣿⣷⣿⣿⣾⎥
⎢⣤⢤⠀⠘⢿⣿⡎⠋⠚⠟⠃⠂⠀⠁⠛⠛⠛⠟⠻⣦⡄⣿⣯⠗⠺⠓⠚⢻⠏⠞⣼⡗⠛⡬⡟⣽⣿⡿⣷⣷⎥
⎢⢩⣿⣤⣼⣬⣿⣤⣤⣿⣤⣤⣤⣤⣤⣤⣤⣤⣤⣤⣭⣿⣿⣧⣼⣤⣼⣤⣤⣧⣤⣿⣧⣿⣿⣼⣯⣯⣼⣿⣿⎥
⎢⢿⣿⣿⣿⣿⡿⢩⣽⢿⣯⡽⠋⢙⣛⣟⣻⡿⣯⢯⠟⣉⣿⣿⣿⣯⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣽⢿⣟⣯⣻⎥
⎢⢸⢼⣿⣿⣿⡿⢜⣵⢨⣾⢿⣧⠺⠛⣷⡩⣯⣿⢾⠂⣀⣿⣯⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡿⣿⣷⡟⣿⣻⢽⎥
⎢⢸⣿⣿⣿⣿⣷⠠⢿⣻⣽⠟⡯⣴⣮⣯⣙⡟⣯⣾⣀⠀⣿⣿⣿⣿⣿⣿⣿⣏⣿⣿⣿⣿⣿⣿⣿⣿⣿⣻⣭⎥
⎢⢹⣽⣿⣿⣿⣿⠚⣭⢥⢭⠥⠻⣿⢮⣿⣿⣖⣾⣫⠅⠉⣿⣿⣿⣿⣿⣯⣽⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⢸⣿⣽⣿⣿⡿⠼⡷⣻⣷⢧⣬⣩⣭⣿⡞⣿⣷⢶⠿⠿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣷⣿⣿⣿⣿⣿⣿⣞⎥
⎢⢸⣿⣾⣿⣿⣿⣺⣕⣾⢟⣶⣶⣶⣯⡩⣿⣙⠾⡛⡤⣿⣿⣿⣿⣿⡿⣿⣿⣿⣿⣽⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⠽⢿⣿⣿⣿⣿⣿⡿⣿⣞⡿⢾⣿⣻⣟⢻⣿⣿⣟⣭⡶⣿⣟⣿⢿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣾⣿⣿⣿⣿⣿⣶⣧⢷⣷⣽⡳⣻⣿⣿⣽⣿⣿⡿⣋⣿⣿⢷⣿⣭⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎣⣿⣿⠿⣿⣿⣿⣿⡮⡿⣾⣯⠶⠝⠏⠿⡵⣻⣿⢽⣿⣿⣿⣯⣻⣟⣞⡟⣾⣿⣿⣻⢿⣿⣿⣿⣿⣿⣿⣿⣿⎦

In [330]:
T1 = C \ ((C + L) * T0.a) - T0.a
T1 = FEM.ScalarField([], reshape(T1, :, 1), [0.0], [], 1, :T)

LowLevelFEM.ScalarField(Matrix{Float64}[], [4.016618420386749; 4.08016298389731; … ; 1.7404374876010706; 0.3703037982343744;;], [0.0], Int64[], 1, :T)

In [331]:
FEM.showDoFResults(thermo, T1, :T, visible=true)

1

In [332]:
FEM.openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XRequest.18: BadValue 0x0


In [333]:
gmsh.finalize()